In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import *
import json
import numpy as np
import string
from datetime import *
import re

In [2]:
df = pd.read_json('all_movies.json').T

In [3]:
df.isna().sum()

title             16
release           29
popularity        16
budget            16
genres            16
overview          16
language          16
votes             16
status            16
cast              16
director          35
screenwriter    1326
producer         423
music           2674
dtype: int64

In [4]:
df.dropna(subset=['title', 'release'], inplace = True)

In [5]:
df.isna().sum()

title              0
release            0
popularity         0
budget             0
genres             0
overview           0
language           0
votes              0
status             0
cast               0
director          12
screenwriter    1299
producer         398
music           2645
dtype: int64

In [6]:
df[['release', 'votes']] = df[['release', 'votes']].astype('int64')
df[['popularity', 'budget']] = df[['popularity','budget']].astype('float64')

In [7]:
df.drop_duplicates(subset=['title'], ignore_index=True, inplace=True)
# Drop instances where title is duplicated. Result of bad choice in search query
df.set_index('title', inplace= True)

In [8]:
df = df[np.logical_not(df['director'].isna() & df['screenwriter'].isna() & df['producer'].isna() & df['music'].isna())]
#Filtered out movies whose director, screenwriter, producer and music director are all not available
df = df[np.logical_not(df['screenwriter'].isna() & df['producer'].isna() & df['music'].isna())]

In [9]:
df = df.dropna(subset=['director'])

In [10]:
df.groupby('status').count()

,release,popularity,budget,genres,overview,language,votes,cast,director,screenwriter,producer,music
status,,,,,,,,,,,,
Post Production,1,1,1,1,1,1,1,1,1,0,1,0
Released,2810,2810,2810,2810,2810,2810,2810,2810,2810,1816,2702,494


In [11]:
df = df[df['status'] == 'Released']

In [12]:
df.groupby('status').count()

,release,popularity,budget,genres,overview,language,votes,cast,director,screenwriter,producer,music
status,,,,,,,,,,,,
Released,2810,2810,2810,2810,2810,2810,2810,2810,2810,1816,2702,494


In [13]:
df = df[df['status'] == 'Released']

In [14]:
def isEmpty(lst):
    if lst == []:
        return True
    return False

In [15]:
exists = []
for cast in df['cast']:
    exists.append(isEmpty(cast))
exists = np.logical_not(exists)
#Movies for which the cast has not been recorded

In [16]:
df = df[exists]

In [17]:
df.head()

,release,popularity,budget,genres,overview,language,votes,status,cast,director,screenwriter,producer,music
title,,,,,,,,,,,,,
All About Eve,1950,18.255,1400000.0,[Drama],From the moment she glimpses her idol at the s...,en,879,Released,"[Bette Davis, Anne Baxter, George Sanders]",Joseph L. Mankiewicz,Joseph L. Mankiewicz,Darryl F. Zanuck,Urban Thielmann
Sunset Boulevard,1950,16.261,1752000.0,[Drama],A hack screenwriter writes a screenplay for a ...,en,1396,Released,"[William Holden, Gloria Swanson, Erich von Str...",Billy Wilder,Billy Wilder,Charles Brackett,None
In a Lonely Place,1950,9.921,0.0,"[Mystery, Thriller]",An aspiring actress begins to suspect that her...,en,269,Released,"[Humphrey Bogart, Gloria Grahame, Frank Lovejoy]",Nicholas Ray,Andrew Solt,Robert Lord,None
A Streetcar Named Desire,1951,13.575,1800000.0,[Drama],Disturbed Blanche DuBois moves in with her sis...,en,744,Released,"[Vivien Leigh, Marlon Brando, Kim Hunter]",Elia Kazan,Tennessee Williams,Charles K. Feldman,None
An American in Paris,1951,11.251,2723903.0,"[Comedy, Drama, Music, Romance]",Jerry Mulligan is an exuberant American expatr...,en,318,Released,"[Gene Kelly, Leslie Caron, Oscar Levant]",Vincente Minnelli,Alan Jay Lerner,Arthur Freed,George Gershwin


In [18]:
df.isna().sum()
df.drop(['screenwriter', 'music', 'producer'], 1, inplace = True)

In [19]:
def search_index(data:pd.DataFrame, query):
    bool_list = []
    for i in list(data.index):
        bool_list.append(query.lower() in i.lower())
    return data.loc[bool_list]

In [20]:
genres = []
for i, n in df.iterrows():
    genres.extend(n['genres'])

In [21]:
genre_dict = {}

In [22]:
for genre in np.unique(genres):
    genre_dict[genre] = []
    for idx, row in df.iterrows():
        genre_dict[genre].append(int(genre in row['genres']))

In [23]:
for genre in genre_dict.keys():
    df[genre] = genre_dict[genre]

In [24]:
y = df[np.unique(genres)]

In [27]:
feature_cols = ['release', 'popularity', 'budget', 'genres', 'overview', 'language',
       'votes', 'status', 'cast', 'director']

In [28]:
X = df[feature_cols]

In [38]:
actors = []
for i, n in df.iterrows():
    actors.extend(n['cast'])